# Horse Racing Bet Builder

This notebook builds comprehensive betting strategies based on predictions from PredictRace.ipynb.

## Prerequisites
- Run PredictRace.ipynb first to generate prediction data
- Ensure the predictions CSV file is available in the current directory

In [51]:
# Import required libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime
import glob

print("📊 Horse Racing Bet Builder")
print("" + "="*50)

📊 Horse Racing Bet Builder


In [52]:
# Load prediction data from CSV
# Look for the most recent predictions file
prediction_files = glob.glob("predictions_*.csv")

if not prediction_files:
    print("❌ No prediction files found. Please run PredictRace.ipynb first.")
    print("   Expected file pattern: predictions_[track]_[date].csv")
    raise FileNotFoundError("No prediction CSV files found")

# Use the most recent file
latest_file = max(prediction_files, key=os.path.getctime)
print(f"📂 Loading predictions from: {latest_file}")

# Load the data
results_df = pd.read_csv(latest_file)

# Extract track and date info from filename
filename_parts = latest_file.replace('.csv', '').split('_')
if len(filename_parts) >= 3:
    track_abbreviation = filename_parts[1]
    race_date = filename_parts[2]
else:
    track_abbreviation = "unknown"
    race_date = "unknown"

# Set track name based on abbreviation
track_names = {
    "cd": "Churchill Downs",
    "aq": "Aqueduct",
    "ct": "Charles Town",
    "ke": "Keeneland",
    "pid": "Presque Isle Downs",
    "sa": "Santa Anita"
}
track_name = track_names.get(track_abbreviation, track_abbreviation.upper())

print(f"✅ Data loaded successfully!")
print(f"📊 {len(results_df)} entries for {len(results_df['race_id'].unique())} races")
print(f"🏁 Track: {track_name}")
print(f"📅 Date: {race_date}")
print(f"" + "="*50)

📂 Loading predictions from: predictions_cd_20250625.csv
✅ Data loaded successfully!
📊 96 entries for 9 races
🏁 Track: Churchill Downs
📅 Date: 20250625


## Betting Configuration

Set your betting parameters here:

In [53]:
# BETTING CONFIGURATION
# ======================
# 🎯 MODIFY THESE PARAMETERS FOR YOUR BETTING STRATEGY

# Basic Settings
base_bet_amount = .2      # Base bet amount in dollars

# Multi-Race Exotic Wager Settings
starting_race_number = 4    # Which race to start multi-race wagers
number_of_races = 6         # How many consecutive races to include

print(f"🎲 BET BUILDER CONFIGURATION")
print(f"{'='*40}")
print(f"Base Bet Amount: ${base_bet_amount:.2f}")
print(f"Multi-Race Sequence: Races {starting_race_number}-{starting_race_number + number_of_races - 1}")
print(f"{'='*40}")

🎲 BET BUILDER CONFIGURATION
Base Bet Amount: $0.20
Multi-Race Sequence: Races 4-9


# Betting Functions

Core functions for analyzing races and building betting strategies.

In [54]:
# Multi-Race Betting Function
def generate_multi_race_bets(start_race, num_races, base_amount=2.00):
    """
    Generate specific betting recommendations for multi-race wagers
    """
    
    # Get all available races from the actual data
    all_race_ids = results_df['race_id'].unique()
    all_races = []
    race_id_map = {}
    
    for race_id in all_race_ids:
        race_num = int(race_id.split('_')[1])
        all_races.append(race_num)
        race_id_map[race_num] = race_id
    
    all_races.sort()
    
    # Validate input parameters
    if start_race not in all_races:
        print(f"❌ ERROR: Race {start_race} not found in today's card")
        print(f"Available races: {all_races}")
        return None, 0
    
    if start_race + num_races - 1 > max(all_races):
        print(f"❌ ERROR: Not enough races available")
        print(f"Requested: Races {start_race}-{start_race + num_races - 1}")
        print(f"Available: Races {min(all_races)}-{max(all_races)}")
        return None, 0
    
    # Get the target races
    target_races = list(range(start_race, start_race + num_races))
    
    print(f"\\n🎯 TARGET RACES: {target_races}")
    print(f"{'='*60}")
    
    # Analyze each target race
    race_analysis = []
    total_confidence_score = 0
    
    for race_num in target_races:
        race_id = race_id_map[race_num]  # Use the actual race ID from the data
        race_data = results_df[results_df['race_id'] == race_id].sort_values('predicted_finish_position')
        
        if len(race_data) == 0:
            print(f"❌ No data for Race {race_num}")
            continue
            
        # Get top picks
        top_pick = race_data.iloc[0]
        second_pick = race_data.iloc[1] if len(race_data) > 1 else None
        
        # Extract horse info
        horse_name = top_pick['horse_id'].split('_')[0]
        prog_num = top_pick['horse_id'].split('_')[1]
        predicted_pos = top_pick['predicted_finish_position']
        
        # Determine confidence level
        if predicted_pos <= 1.5:
            confidence = "HIGH"
            confidence_score = 3
        elif predicted_pos <= 2.5:
            confidence = "MODERATE"
            confidence_score = 2
        else:
            confidence = "LOW"
            confidence_score = 1
            
        total_confidence_score += confidence_score
        
        # Second pick info
        second_info = ""
        spread = 0
        if second_pick is not None:
            second_horse = second_pick['horse_id'].split('_')[0]
            second_prog = second_pick['horse_id'].split('_')[1]
            spread = second_pick['predicted_finish_position'] - predicted_pos
            second_info = f"#{second_prog} {second_horse} (Pred: {second_pick['predicted_finish_position']:.2f})"
        
        race_analysis.append({
            'race_num': race_num,
            'race_id': race_id,
            'top_pick': f"#{prog_num} {horse_name}",
            'predicted_pos': predicted_pos,
            'confidence': confidence,
            'confidence_score': confidence_score,
            'second_pick': second_info,
            'spread': spread,
            'field_size': len(race_data)
        })
        
        print(f"\\n🏁 RACE {race_num} - {confidence} CONFIDENCE")
        print(f"   Top Pick: #{prog_num} {horse_name} (Predicted: {predicted_pos:.2f})")
        if second_pick is not None:
            print(f"   2nd Pick: {second_info} (Spread: {spread:.2f})")
        print(f"   Field Size: {len(race_data)} horses")
    
    return race_analysis, total_confidence_score

In [55]:
# All-Races Analysis Function
def analyze_all_races(base_amount=2.00):
    """
    Analyze all races for win bets and exacta opportunities
    """
    
    # Get all available races and analyze each one
    all_race_ids = results_df['race_id'].unique()
    all_races_data = []

    for race_id in sorted(all_race_ids):
        race_num = int(race_id.split('_')[1])
        race_data = results_df[results_df['race_id'] == race_id].sort_values('predicted_finish_position')
        
        if len(race_data) == 0:
            continue
            
        # Get top picks
        top_pick = race_data.iloc[0]
        second_pick = race_data.iloc[1] if len(race_data) > 1 else None
        third_pick = race_data.iloc[2] if len(race_data) > 2 else None
        
        # Extract horse info
        top_horse = top_pick['horse_id'].split('_')[0]
        top_prog = top_pick['horse_id'].split('_')[1]
        top_pred = top_pick['predicted_finish_position']
        
        # Determine confidence level
        if top_pred <= 1.5:
            confidence = "HIGH"
            win_confidence = "STRONG"
        elif top_pred <= 2.5:
            confidence = "MODERATE" 
            win_confidence = "GOOD"
        else:
            confidence = "LOW"
            win_confidence = "WEAK"
        
        # Exacta analysis
        exacta_suitable = False
        exacta_info = ""
        spread = 0
        
        if second_pick is not None:
            second_horse = second_pick['horse_id'].split('_')[0]
            second_prog = second_pick['horse_id'].split('_')[1]
            second_pred = second_pick['predicted_finish_position']
            spread = second_pred - top_pred
            
            if spread < 1.0:  # Close race - good for exacta
                exacta_suitable = True
                exacta_info = f"#{top_prog}-{second_prog} Box (Spread: {spread:.2f})"
            else:
                exacta_info = f"Wide spread ({spread:.2f}) - Consider top pick only"
        
        all_races_data.append({
            'race_num': race_num,
            'race_id': race_id,
            'top_pick': f"#{top_prog} {top_horse}",
            'predicted_pos': top_pred,
            'confidence': confidence,
            'win_confidence': win_confidence,
            'exacta_suitable': exacta_suitable,
            'exacta_info': exacta_info,
            'spread': spread,
            'field_size': len(race_data),
            'second_pick': f"#{second_prog} {second_horse}" if second_pick is not None else None,
            'third_pick': f"#{third_pick['horse_id'].split('_')[1]} {third_pick['horse_id'].split('_')[0]}" if third_pick is not None else None
        })
    
    return all_races_data

In [56]:
# Exotic Wager Builder Function
def build_exotic_wagers(start_race, num_races, base_amount=2.00):
    """
    Build exotic multi-race wagers with dynamic horse selection based on confidence
    """
    
    # Get the specified races for exotic wagers
    target_races = list(range(start_race, start_race + num_races))
    
    # Get all races data first
    all_races_data = analyze_all_races(base_amount)
    
    # Find data for target races
    target_race_data = []
    for race_num in target_races:
        race_info = next((r for r in all_races_data if r['race_num'] == race_num), None)
        if race_info:
            target_race_data.append(race_info)
    
    if len(target_race_data) < 2:
        print("❌ Need at least 2 races for exotic wagers")
        return None, None
    
    print(f"\\n📋 RACE ANALYSIS FOR EXOTIC WAGERS:")
    print(f"{'-'*50}")
    
    # Analyze each race and determine how many horses to use
    exotic_selections = []
    
    for race_data in target_race_data:
        race_num = race_data['race_num']
        confidence = race_data['confidence']
        spread = race_data['spread']
        
        # Determine how many horses to use based on confidence and race competitiveness
        if confidence == 'HIGH' and spread > 0.8:
            # High confidence, not close race - use 1 horse (key)
            num_horses = 1
            selection_type = "KEY"
            reason = "High confidence, clear favorite"
            
        elif confidence == 'HIGH' and spread <= 0.8:
            # High confidence but close race - use 2 horses
            num_horses = 2
            selection_type = "SMALL"
            reason = "High confidence but competitive"
            
        elif confidence == 'MODERATE' and spread <= 0.7:
            # Moderate confidence, close race - use 3 horses
            num_horses = 3
            selection_type = "MEDIUM"
            reason = "Moderate confidence, very competitive"
            
        elif confidence == 'MODERATE':
            # Moderate confidence, wider spread - use 2 horses
            num_horses = 2
            selection_type = "SMALL"
            reason = "Moderate confidence"
            
        else:
            # Low confidence - use 3-4 horses for coverage
            num_horses = 4 if race_data['field_size'] >= 8 else 3
            selection_type = "LARGE"
            reason = "Low confidence, need coverage"
        
        # Don't exceed field size
        num_horses = min(num_horses, race_data['field_size'])
        
        exotic_selections.append({
            'race_num': race_num,
            'num_horses': num_horses,
            'selection_type': selection_type,
            'reason': reason,
            'confidence': confidence,
            'spread': spread,
            'top_pick': race_data['top_pick'],
            'second_pick': race_data['second_pick'],
            'third_pick': race_data['third_pick']
        })
        
        print(f"Race {race_num}: Use {num_horses} horses ({selection_type}) - {reason}")
        print(f"   Confidence: {confidence}, Spread: {spread:.2f}")
        
        # Show which horses
        horses_to_show = []
        horses_to_show.append(race_data['top_pick'])
        if num_horses >= 2 and race_data['second_pick']:
            horses_to_show.append(race_data['second_pick'])
        if num_horses >= 3 and race_data['third_pick']:
            horses_to_show.append(race_data['third_pick'])
        if num_horses == 4:
            # Get 4th choice
            race_full_data = results_df[results_df['race_id'] == race_data['race_id']].sort_values('predicted_finish_position')
            if len(race_full_data) >= 4:
                fourth_pick = race_full_data.iloc[3]
                fourth_horse = f"#{fourth_pick['horse_id'].split('_')[1]} {fourth_pick['horse_id'].split('_')[0]}"
                horses_to_show.append(fourth_horse)
        
        print(f"   Horses: {', '.join(horses_to_show)}")
        print()
    
    return exotic_selections, target_race_data

# Betting Strategy Execution

Execute your betting strategies based on the configuration above.

In [57]:
# ALL-RACES WIN & EXACTA ANALYSIS
print(f"\\n{'🏆 ALL RACES WIN & EXACTA OPPORTUNITIES'}")
print(f"{'='*70}")

# Get all races data
all_races_data = analyze_all_races(base_bet_amount)

# Display all races analysis
for race_data in all_races_data:
    print(f"\\n🏁 RACE {race_data['race_num']} - {race_data['win_confidence']} WIN CONFIDENCE")
    print(f"   Top Pick: {race_data['top_pick']} (Predicted: {race_data['predicted_pos']:.2f})")
    print(f"   Field Size: {race_data['field_size']} horses")
    
    # Win bet recommendation
    win_bet_size = base_bet_amount
    if race_data['confidence'] == 'HIGH':
        win_bet_size *= 1.5
    elif race_data['confidence'] == 'MODERATE':
        win_bet_size *= 1.2
    
    print(f"   💰 Win Bet: ${win_bet_size:.2f} ({race_data['confidence']} confidence)")
    
    # Exacta recommendation
    if race_data['exacta_suitable']:
        exacta_cost = base_bet_amount * 2  # $2 box
        print(f"   🎯 Exacta Box: {race_data['exacta_info']} - ${exacta_cost:.2f}")
        print(f"       Top 2: {race_data['top_pick']} / {race_data['second_pick']}")
    else:
        print(f"   ⚠️  Exacta: {race_data['exacta_info']}")

# Summary of recommendations
total_win_cost = sum(base_bet_amount * (1.5 if r['confidence'] == 'HIGH' else 1.2 if r['confidence'] == 'MODERATE' else 1.0) for r in all_races_data)
total_exacta_cost = sum(base_bet_amount * 2 for r in all_races_data if r['exacta_suitable'])
high_confidence_races = len([r for r in all_races_data if r['confidence'] == 'HIGH'])
exacta_opportunities = len([r for r in all_races_data if r['exacta_suitable']])

print(f"\n{'📊 ALL-RACES BETTING SUMMARY'}")
print(f"{'='*50}")
print(f"Total Races: {len(all_races_data)}")
print(f"High Confidence Races: {high_confidence_races}")
print(f"Exacta Opportunities: {exacta_opportunities}")
print(f"Total Win Bets Cost: ${total_win_cost:.2f}")
print(f"Total Exacta Boxes Cost: ${total_exacta_cost:.2f}")
print(f"Combined Investment: ${total_win_cost + total_exacta_cost:.2f}")

\n🏆 ALL RACES WIN & EXACTA OPPORTUNITIES
\n🏁 RACE 1 - WEAK WIN CONFIDENCE
   Top Pick: #1 JUST AN OPINION (Predicted: 2.58)
   Field Size: 6 horses
   💰 Win Bet: $0.20 (LOW confidence)
   🎯 Exacta Box: #1-5 Box (Spread: 0.03) - $0.40
       Top 2: #1 JUST AN OPINION / #5 COPPER EM
\n🏁 RACE 2 - WEAK WIN CONFIDENCE
   Top Pick: #8 ACTIVITY (Predicted: 2.65)
   Field Size: 10 horses
   💰 Win Bet: $0.20 (LOW confidence)
   ⚠️  Exacta: Wide spread (1.69) - Consider top pick only
\n🏁 RACE 3 - WEAK WIN CONFIDENCE
   Top Pick: #4 AMERICAN LAW (Predicted: 3.31)
   Field Size: 8 horses
   💰 Win Bet: $0.20 (LOW confidence)
   🎯 Exacta Box: #4-7 Box (Spread: 0.01) - $0.40
       Top 2: #4 AMERICAN LAW / #7 REFUAH
\n🏁 RACE 4 - WEAK WIN CONFIDENCE
   Top Pick: #9 COMMAND THE STARS (Predicted: 4.97)
   Field Size: 13 horses
   💰 Win Bet: $0.20 (LOW confidence)
   🎯 Exacta Box: #9-4 Box (Spread: 0.44) - $0.40
       Top 2: #9 COMMAND THE STARS / #4 WAKUDA
\n🏁 RACE 5 - WEAK WIN CONFIDENCE
   Top Pick: 

In [58]:
# MULTI-RACE BETTING RECOMMENDATIONS
print(f"\\n🎲 MULTI-RACE BETTING RECOMMENDATIONS")
print(f"{'='*50}")
print(f"Starting Race: {starting_race_number}")
print(f"Number of Races: {number_of_races}")
print(f"Base Bet Amount: ${base_bet_amount:.2f}")
print(f"{'='*50}")

# Generate multi-race analysis
race_analysis, confidence_score = generate_multi_race_bets(starting_race_number, number_of_races, base_bet_amount)

if race_analysis:
    # Calculate overall strategy confidence
    avg_confidence = confidence_score / len(race_analysis)
    overall_confidence = "HIGH" if avg_confidence >= 2.5 else "MODERATE" if avg_confidence >= 2.0 else "LOW"
    
    print(f"\\nOverall Confidence Level: {overall_confidence} (Score: {confidence_score}/{len(race_analysis)*3})")
    
    # Strategy 1: WIN PARLAY (All top picks to win)
    print(f"\\n📊 STRATEGY 1: WIN PARLAY")
    print(f"{'-'*40}")
    win_parlay_horses = [f"Race {r['race_num']}: {r['top_pick']}" for r in race_analysis]
    
    # Calculate expected probability based on historical performance
    high_conf_races = sum(1 for r in race_analysis if r['confidence'] == 'HIGH')
    mod_conf_races = sum(1 for r in race_analysis if r['confidence'] == 'MODERATE')
    low_conf_races = sum(1 for r in race_analysis if r['confidence'] == 'LOW')
    
    # Use historical win rates by confidence level
    estimated_win_prob = ((high_conf_races * 0.80) + (mod_conf_races * 0.65) + (low_conf_races * 0.45)) / len(race_analysis)
    parlay_success_prob = estimated_win_prob ** len(race_analysis)
    
    for horse in win_parlay_horses:
        print(f"   • {horse}")
    
    print(f"   Cost: ${base_bet_amount:.2f}")
    print(f"   Estimated Success Rate: {parlay_success_prob:.1%}")
    print(f"   Recommendation: {'✅ RECOMMENDED' if overall_confidence in ['HIGH', 'MODERATE'] and len(race_analysis) <= 3 else '⚠️  RISKY - Consider alternatives'}")
    
    # Strategy 2: EXACTA BOXES (Where spreads are small)
    print(f"\\n📊 STRATEGY 2: EXACTA BOXES")
    print(f"{'-'*40}")
    
    exacta_races = [r for r in race_analysis if r['spread'] < 1.0 and r['second_pick']]
    exacta_cost = 0
    
    if exacta_races:
        for race in exacta_races:
            cost_per_race = base_bet_amount * 2  # $1 box = $2 total
            exacta_cost += cost_per_race
            print(f"   Race {race['race_num']}: {race['top_pick']} / {race['second_pick'].split(' (')[0]}")
            print(f"     Box Cost: ${cost_per_race:.2f} (Spread: {race['spread']:.2f})")
        
        print(f"   Total Cost: ${exacta_cost:.2f}")
        print(f"   Estimated Coverage: ~75% chance per race")
        print(f"   Recommendation: ✅ GOOD VALUE - Close predictions suggest competitive races")
    else:
        print(f"   No suitable races (need spread < 1.0)")
        print(f"   Recommendation: Consider win bets instead")
    
    # Strategy 3: PROGRESSIVE WIN BETS
    print(f"\\n📊 STRATEGY 3: PROGRESSIVE WIN BETS")
    print(f"{'-'*40}")
    
    progressive_cost = 0
    current_bet = base_bet_amount
    
    for i, race in enumerate(race_analysis):
        progressive_cost += current_bet
        print(f"   Race {race['race_num']}: {race['top_pick']} - ${current_bet:.2f}")
        
        # Increase bet size for higher confidence races
        if race['confidence'] == 'HIGH':
            current_bet *= 1.2
        elif race['confidence'] == 'MODERATE':
            current_bet *= 1.1
    
    print(f"   Total Cost: ${progressive_cost:.2f}")
    print(f"   Strategy: Increase bets on higher confidence races")
    print(f"   Recommendation: ✅ SAFEST - Win each race independently")
    
    # FINAL RECOMMENDATION
    print(f"\\n🎯 FINAL RECOMMENDATION")
    print(f"{'='*60}")
    
    if overall_confidence == 'HIGH' and len(race_analysis) <= 3:
        recommended_strategy = "Strategy 1 (Win Parlay)"
        recommended_cost = base_bet_amount
    elif exacta_races and overall_confidence in ['HIGH', 'MODERATE']:
        recommended_strategy = "Strategy 2 (Exacta Boxes)"
        recommended_cost = exacta_cost
    else:
        recommended_strategy = "Strategy 3 (Progressive Win Bets)"
        recommended_cost = progressive_cost
    
    print(f"✅ RECOMMENDED: {recommended_strategy}")
    print(f"💰 Total Investment: ${recommended_cost:.2f}")
    print(f"🎯 Risk Level: {overall_confidence}")
    
    # Risk warnings
    if len(race_analysis) > 4:
        print(f"⚠️  WARNING: {len(race_analysis)} races is high risk - consider reducing")
    if overall_confidence == 'LOW':
        print(f"⚠️  WARNING: Low confidence - consider smaller bets or skip")

else:
    print("❌ Could not generate recommendations - check race parameters")

\n🎲 MULTI-RACE BETTING RECOMMENDATIONS
Starting Race: 4
Number of Races: 6
Base Bet Amount: $0.20
\n🎯 TARGET RACES: [4, 5, 6, 7, 8, 9]
\n🏁 RACE 4 - LOW CONFIDENCE
   Top Pick: #9 COMMAND THE STARS (Predicted: 4.97)
   2nd Pick: #4 WAKUDA (Pred: 5.41) (Spread: 0.44)
   Field Size: 13 horses
\n🏁 RACE 5 - LOW CONFIDENCE
   Top Pick: #1 PANCAKE HOUSE (Predicted: 4.90)
   2nd Pick: #11 KIGALI (Pred: 5.86) (Spread: 0.96)
   Field Size: 14 horses
\n🏁 RACE 6 - LOW CONFIDENCE
   Top Pick: #4 AMELIA'S ECHO (Predicted: 2.98)
   2nd Pick: #3 RISUKO (Pred: 3.60) (Spread: 0.62)
   Field Size: 11 horses
\n🏁 RACE 7 - LOW CONFIDENCE
   Top Pick: #3 LIAM THE BRAVE (Predicted: 3.93)
   2nd Pick: #1 METATRON'S MUSE (Pred: 4.13) (Spread: 0.20)
   Field Size: 10 horses
\n🏁 RACE 8 - LOW CONFIDENCE
   Top Pick: #4 AJAAYB (Predicted: 3.51)
   2nd Pick: #9 RISK IT (Pred: 3.77) (Spread: 0.26)
   Field Size: 9 horses
\n🏁 RACE 9 - LOW CONFIDENCE
   Top Pick: #13 OCEAN RIPPLE (Predicted: 4.55)
   2nd Pick: #10 WILD

In [ ]:
# EXOTIC MULTI-RACE WAGERS CONSTRUCTION
print(f"\n{'🎰 EXOTIC MULTI-RACE WAGERS'}")
print(f"{'='*70}")
print(f"Building exotic wagers for Races {starting_race_number}-{starting_race_number + number_of_races - 1}")

# Build the exotic wager selections
exotic_selections, target_race_data = build_exotic_wagers(starting_race_number, number_of_races, base_bet_amount)

if exotic_selections and len(exotic_selections) >= 2:
    
    # Calculate costs for different exotic wagers
    print(f"\\n{'💰 EXOTIC WAGER COSTS & RECOMMENDATIONS'}")
    print(f"{'='*60}")
    
    # Store exotic costs for summary
    exotic_costs = {}
    
    # Daily Double (first 2 races)
    if len(exotic_selections) >= 2:
        dd_combinations = exotic_selections[0]['num_horses'] * exotic_selections[1]['num_horses']
        dd_cost = dd_combinations * base_bet_amount
        exotic_costs['Daily Double'] = dd_cost
        print(f"\\n🎯 DAILY DOUBLE (Races {exotic_selections[0]['race_num']}-{exotic_selections[1]['race_num']}):")
        print(f"   Combinations: {exotic_selections[0]['num_horses']} x {exotic_selections[1]['num_horses']} = {dd_combinations}")
        print(f"   Cost: ${dd_cost:.2f} (${base_bet_amount:.2f} per combination)")
        print(f"   Strategy: {exotic_selections[0]['selection_type']} in Race {exotic_selections[0]['race_num']}, {exotic_selections[1]['selection_type']} in Race {exotic_selections[1]['race_num']}")
    
    # Pick 3 (if 3+ races)
    if len(exotic_selections) >= 3:
        p3_combinations = exotic_selections[0]['num_horses'] * exotic_selections[1]['num_horses'] * exotic_selections[2]['num_horses']
        p3_cost = p3_combinations * base_bet_amount
        exotic_costs['Pick 3'] = p3_cost
        print(f"\\n🎯 PICK 3 (Races {exotic_selections[0]['race_num']}-{exotic_selections[2]['race_num']}):")
        print(f"   Combinations: {exotic_selections[0]['num_horses']} x {exotic_selections[1]['num_horses']} x {exotic_selections[2]['num_horses']} = {p3_combinations}")
        print(f"   Cost: ${p3_cost:.2f} (${base_bet_amount:.2f} per combination)")
        
        # Show strategy
        strategies = [f"{sel['selection_type']} in Race {sel['race_num']}" for sel in exotic_selections[:3]]
        print(f"   Strategy: {', '.join(strategies)}")
    
    # Pick 4 (if 4+ races)
    if len(exotic_selections) >= 4:
        p4_combinations = 1
        for sel in exotic_selections[:4]:
            p4_combinations *= sel['num_horses']
        p4_cost = p4_combinations * base_bet_amount
        exotic_costs['Pick 4'] = p4_cost
        print(f"\\n🎯 PICK 4 (Races {exotic_selections[0]['race_num']}-{exotic_selections[3]['race_num']}):")
        print(f"   Combinations: {' x '.join([str(sel['num_horses']) for sel in exotic_selections[:4]])} = {p4_combinations}")
        print(f"   Cost: ${p4_cost:.2f} (${base_bet_amount:.2f} per combination)")
    
    # Recommendations based on cost and confidence
    print(f"\\n{'🎯 EXOTIC WAGER RECOMMENDATIONS'}")
    print(f"{'='*50}")
    
    high_conf_count = sum(1 for sel in exotic_selections if sel['confidence'] == 'HIGH')
    total_confidence = len([sel for sel in exotic_selections if sel['confidence'] in ['HIGH', 'MODERATE']])
    
    if len(exotic_selections) >= 2 and exotic_costs.get('Daily Double', 0) <= base_bet_amount * 10:
        print(f"✅ DAILY DOUBLE: Cost-effective at ${exotic_costs['Daily Double']:.2f}")
    
    if len(exotic_selections) >= 3 and exotic_costs.get('Pick 3', 0) <= base_bet_amount * 20 and high_conf_count >= 1:
        print(f"✅ PICK 3: Good value at ${exotic_costs['Pick 3']:.2f} with {high_conf_count} high-confidence races")
    
    if len(exotic_selections) >= 4 and exotic_costs.get('Pick 4', 0) <= base_bet_amount * 50 and total_confidence >= 3:
        print(f"✅ PICK 4: Reasonable at ${exotic_costs['Pick 4']:.2f} with good overall confidence")
    elif len(exotic_selections) >= 4:
        print(f"⚠️  PICK 4: High cost (${exotic_costs.get('Pick 4', 0):.2f}) - consider smaller ticket")

else:
    print("❌ Need at least 2 races for exotic wagers")
    exotic_costs = {}

\n🎰 EXOTIC MULTI-RACE WAGERS
Building exotic wagers for Races 4-9
\n📋 RACE ANALYSIS FOR EXOTIC WAGERS:
--------------------------------------------------
Race 4: Use 4 horses (LARGE) - Low confidence, need coverage
   Confidence: LOW, Spread: 0.44
   Horses: #9 COMMAND THE STARS, #4 WAKUDA, #5 JOKE MAKER, #3 AMES

Race 5: Use 4 horses (LARGE) - Low confidence, need coverage
   Confidence: LOW, Spread: 0.96
   Horses: #1 PANCAKE HOUSE, #11 KIGALI, #6 DBOOK, #9 D DAY REUNION

Race 6: Use 4 horses (LARGE) - Low confidence, need coverage
   Confidence: LOW, Spread: 0.62
   Horses: #4 AMELIA'S ECHO, #3 RISUKO, #1 ADULARE, #8 TIMBO'S MISCHIEF

Race 7: Use 4 horses (LARGE) - Low confidence, need coverage
   Confidence: LOW, Spread: 0.20
   Horses: #3 LIAM THE BRAVE, #1 METATRON'S MUSE, #9 COPPERLITE, #2 NIKOLAOU

Race 8: Use 4 horses (LARGE) - Low confidence, need coverage
   Confidence: LOW, Spread: 0.26
   Horses: #4 AJAAYB, #9 RISK IT, #7 FURIOUS ANGER, #6 NORTHERN CHILL

Race 9: Use 4 hor

In [60]:
# COMPREHENSIVE BETTING STRATEGY SUMMARY
print(f"\\n{'🎯 COMPLETE BETTING STRATEGY SUMMARY'}")
print(f"{'='*70}")

# Calculate total investments for different approaches
all_races_investment = total_win_cost + total_exacta_cost

print(f"\\n💰 INVESTMENT OPTIONS COMPARISON:")
print(f"{'-'*50}")

# Option 1: All races win + exacta
print(f"📋 OPTION 1: ALL RACES WIN & EXACTA")
print(f"   Win bets on all {len(all_races_data)} races: ${total_win_cost:.2f}")
print(f"   Exacta boxes on {exacta_opportunities} races: ${total_exacta_cost:.2f}")
print(f"   Total Investment: ${all_races_investment:.2f}")
print(f"   Strategy: Comprehensive coverage, moderate risk")

# Option 2: Specified races exotic wagers
if 'exotic_costs' in locals() and exotic_costs:
    print(f"\\n📋 OPTION 2: MULTI-RACE EXOTICS (Races {starting_race_number}-{starting_race_number + number_of_races - 1})")
    for bet_type, cost in exotic_costs.items():
        print(f"   {bet_type}: ${cost:.2f}")
    
    min_exotic_cost = min(exotic_costs.values())
    max_exotic_cost = max(exotic_costs.values())
    print(f"   Investment Range: ${min_exotic_cost:.2f} - ${max_exotic_cost:.2f}")
    print(f"   Strategy: High-risk, high-reward exotic wagers")

# Option 3: Combination approach
if 'exotic_costs' in locals() and exotic_costs:
    # Suggest win bets on high-confidence races + one exotic wager
    high_conf_win_cost = sum(base_bet_amount * 1.5 for r in all_races_data if r['confidence'] == 'HIGH')
    recommended_exotic = min(exotic_costs.items(), key=lambda x: x[1])  # Cheapest exotic
    combo_cost = high_conf_win_cost + recommended_exotic[1]
    
    print(f"\\n📋 OPTION 3: COMBINATION APPROACH")
    print(f"   Win bets on {high_confidence_races} high-confidence races: ${high_conf_win_cost:.2f}")
    print(f"   {recommended_exotic[0]}: ${recommended_exotic[1]:.2f}")
    print(f"   Total Investment: ${combo_cost:.2f}")
    print(f"   Strategy: Balanced risk, diversified approach")

# Final recommendation based on bankroll and risk tolerance
print(f"\\n🎯 RECOMMENDED STRATEGY:")
print(f"{'-'*30}")

if 'exotic_costs' in locals() and exotic_costs:
    # If we have affordable exotic options
    daily_double_cost = exotic_costs.get('Daily Double', float('inf'))
    pick3_cost = exotic_costs.get('Pick 3', float('inf'))
    
    if daily_double_cost <= base_bet_amount * 6:  # Reasonable DD cost
        print(f"✅ CONSERVATIVE: Daily Double (${daily_double_cost:.2f}) + Win bets on high-confidence races")
        print(f"   Total: ${daily_double_cost + high_conf_win_cost:.2f}")
    elif pick3_cost <= base_bet_amount * 20:  # Reasonable P3 cost
        print(f"✅ MODERATE: Pick 3 (${pick3_cost:.2f}) for big score potential")
    else:
        print(f"✅ CONSERVATIVE: Focus on win bets and exacta boxes")
        print(f"   High-confidence wins + exactas: ${high_conf_win_cost + (exacta_opportunities * base_bet_amount * 2):.2f}")
else:
    print(f"✅ FOCUS ON: Win bets and exacta boxes")
    print(f"   Total investment: ${all_races_investment:.2f}")

print(f"\\n💡 TIPS:")
print(f"   • Start with smaller amounts to test the strategy")
print(f"   • Focus on races with HIGH confidence ratings")
print(f"   • Exacta boxes work best when spread < 1.0")
print(f"   • Multi-race exotics are high-risk, high-reward")
print(f"   • Adjust base_bet_amount to fit your bankroll")

print(f"\\n🔄 TO MODIFY: Change the configuration parameters above and re-run!")
print(f"   - starting_race_number: {starting_race_number}")
print(f"   - number_of_races: {number_of_races}")
print(f"   - base_bet_amount: ${base_bet_amount:.2f}")

print(f"\n💡 Change the parameters in the configuration section and re-run to test different combinations!")

\n🎯 COMPLETE BETTING STRATEGY SUMMARY
\n💰 INVESTMENT OPTIONS COMPARISON:
--------------------------------------------------
📋 OPTION 1: ALL RACES WIN & EXACTA
   Win bets on all 9 races: $1.80
   Exacta boxes on 8 races: $3.20
   Total Investment: $5.00
   Strategy: Comprehensive coverage, moderate risk
\n📋 OPTION 2: MULTI-RACE EXOTICS (Races 4-9)
   Daily Double: $3.20
   Pick 3: $12.80
   Pick 4: $51.20
   Investment Range: $3.20 - $51.20
   Strategy: High-risk, high-reward exotic wagers
\n📋 OPTION 3: COMBINATION APPROACH
   Win bets on 0 high-confidence races: $0.00
   Daily Double: $3.20
   Total Investment: $3.20
   Strategy: Balanced risk, diversified approach
\n🎯 RECOMMENDED STRATEGY:
------------------------------
✅ CONSERVATIVE: Focus on win bets and exacta boxes
   High-confidence wins + exactas: $3.20
\n💡 TIPS:
   • Start with smaller amounts to test the strategy
   • Focus on races with HIGH confidence ratings
   • Exacta boxes work best when spread < 1.0
   • Multi-race ex